In [1]:
import psycopg2
from config import load_config
import pandas as pd
import psycopg2.extras

In [2]:
DATA_DIR = "C:/Projects/Rajk/prog_2/fourth_hw/c4-frontend-barab/data"

In [3]:
DATA_DIR

'C:/Projects/Rajk/prog_2/fourth_hw/c4-frontend-barab/data'

In [4]:
config = load_config()

In [5]:
with psycopg2.connect(**config) as conn:
    print('Connected to the PostgreSQL server.')
    

Connected to the PostgreSQL server.


## Parental guides

In [6]:
data = pd.read_csv("{}/parental-guides.csv".format(DATA_DIR))

In [7]:
data["x"] = data["x"].str.replace(',','')
data["y"] = data["y"].str.replace(',','')

In [8]:
data = data.astype({"x":int})
data = data.astype({"y":int})
data = data.astype({"cat":str})

In [9]:
data.head()

,t,cat,level,x,y,n
0,tt0944947,Sex & Nudity,Severe,7645,8721,7645
1,tt0944947,Violence & Gore,Severe,4106,4471,4106
2,tt0944947,Profanity,Severe,3459,4128,3459
3,tt0944947,"Alcohol, Drugs & Smoking",Moderate,1769,4708,1769
4,tt0944947,Frightening & Intense Scenes,Severe,3163,3889,3163


In [10]:
sql = '''CREATE TABLE parentalguides ( 
t char(20) ,cat char(30),level char(20),x int, y int, n int);'''
delete_sql = '''DROP TABLE parentalguides'''

In [11]:
conn = psycopg2.connect(**config)
cursor = conn.cursor()
cursor.execute(delete_sql)
cursor.execute(sql)
conn.commit()
conn.close()

In [12]:
data.dtypes

t        object
cat      object
level    object
x         int32
y         int32
n         int64
dtype: object

In [13]:
conn = psycopg2.connect(**config)
df_columns = list(data)
# create (col1,col2,...)
columns = ",".join(df_columns)

# create VALUES('%s', '%s",...) one '%s' per column
values = "VALUES({})".format(",".join(["%s" for _ in df_columns])) 

#create INSERT INTO table (columns) VALUES('%s',...)
insert_stmt = "INSERT INTO {} ({}) {}".format("parentalguides",columns,values)

cur = conn.cursor()
psycopg2.extras.execute_batch(cur, insert_stmt, data.values)
conn.commit()
cur.close()

## Series data

In [22]:
data = pd.read_csv("{}/series.csv".format(DATA_DIR))

In [23]:
data.head()

,t,title,rating,vote_num
0,tt0944947,Game of Thrones,9.2,2278227
1,tt8772296,Euphoria,8.3,240171
2,tt7767422,Sex Education,8.3,350179
3,tt1190634,The Boys,8.7,646277
4,tt5180504,The Witcher,8.0,568491


In [15]:
sql = '''CREATE TABLE series ( 
t char(20) ,title char(50),rating float(5),vote_num int);'''
delete_sql = '''DROP TABLE series'''

In [16]:
conn = psycopg2.connect(**config)
cursor = conn.cursor()
cursor.execute(delete_sql)
cursor.execute(sql)
conn.commit()
conn.close()

In [17]:
conn = psycopg2.connect(**config)
df_columns = list(data)
# create (col1,col2,...)
columns = ",".join(df_columns)

# create VALUES('%s', '%s",...) one '%s' per column
values = "VALUES({})".format(",".join(["%s" for _ in df_columns])) 

#create INSERT INTO table (columns) VALUES('%s',...)
insert_stmt = "INSERT INTO {} ({}) {}".format("series",columns,values)

cur = conn.cursor()
psycopg2.extras.execute_batch(cur, insert_stmt, data.values)
conn.commit()
cur.close()

## Set up keys and connections

In [18]:
sql = '''ALTER TABLE series 
ADD PRIMARY KEY (t)'''

In [19]:
conn = psycopg2.connect(**config)
cursor = conn.cursor()
cursor.execute(sql)
conn.commit()
conn.close()


In [26]:
sql = '''ALTER TABLE parentalguides 
ADD FOREIGN KEY (t) REFERENCES series(t)'''

In [27]:
sql

'ALTER TABLE parentalguides \nADD FOREIGN KEY (t) REFERENCES series(t)'

In [21]:
conn = psycopg2.connect(**config)
cursor = conn.cursor()
cursor.execute(sql)
conn.commit()
conn.close()

## Basic query (find the series that satisfy the conditions)

In [60]:
inputs = ["'Sex & Nudity'","'Severe'","'Violence & Gore'","'Severe'"]
sql = ''' 
select s.t, s.title
from series s
join parentalguides pg on s.t = pg.t
where (cat = {} and level = {})
or (cat = {} and level = {})
group by s.t, s.title
having count(distinct cat) = 2
'''.format(*inputs)

In [61]:
conn = psycopg2.connect(**config)
cursor = conn.cursor()
cursor.execute(sql)
rows = cursor.fetchall()
for row in rows:
    print(row[1])
conn.commit()
conn.close()

Oz                                                
South Park                                        
The Sopranos                                      
Berserk                                           
Deadwood                                          
Nip/Tuck                                          
Rome                                              
Brotherhood                                       
Westworld                                         
The Tudors                                        
True Blood                                        
Game of Thrones                                   
Boardwalk Empire                                  
Underbelly                                        
House of the Dragon                               
Mayor of Kingstown                                
Power Book III: Raising Kanan                     
Power Book IV: Force                              
The Boys                                          
Peacemaker                     

In [62]:
len(rows)

58